# Simulation of M9 medium with and without Casamino acids
*Author: Famke Baeuerle*

In [1]:
# imports
import cobra
from libsbml import *
import refinegems as rg
import pandas as pd
import numpy as np

In [2]:
def enhanced_growth(model, base_medium, new_metabs):
    local_medium = base_medium
    with model:
        medium = model.medium
        for met in new_metabs:
            local_medium[met] = 10.0
        model.medium = local_medium
        sol = model.optimize()
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate with additives: ' + str(round(base_growth, 4)))
    print('Doubling time with additives: ' + str(round(dt, 2)) + ' min')

def find_additives(model, base_medium):
    with model:
        medium = model.medium
        model.medium = base_medium
        sol = model.optimize()      
    base_growth = sol.objective_value
    dt = (np.log(2) / sol.objective_value) * 60
    print('Growth rate without additives: ' + str(round(base_growth, 4)))
    print('Doubling time without additives: ' + str(round(dt, 2)) + ' min')
        
    enhancement = {}
    for ex in list(model.exchanges):
        if ex.id not in base_medium.keys():
            with model:
                medium = model.medium
                base_medium.update({ex.id:10.0})
                model.medium = base_medium
                sol = model.optimize()
                if sol.objective_value > base_growth:
                    enhancement[ex.id] = sol.objective_value - base_growth
                base_medium.pop(ex.id)
    
    enh = pd.DataFrame(enhancement.items(), columns=['exchange', 'diff']).sort_values(by=['diff'], ascending=False)
    
    return enh

In [9]:
CasA = pd.read_csv('../../data/media_db.csv')[pd.read_csv('../../data/media_db.csv')['medium']=='CasA']['BiGG'].to_list()
M9 = pd.read_csv('../../data/media_db.csv')[pd.read_csv('../../data/media_db.csv')['medium']=='M9']['BiGG'].to_list()
CasA_med = {'EX_'+i+'_e': 10.0 for i in CasA}
M9_med = {'EX_'+i+'_e': 10.0 for i in M9}
if (M9_med['EX_o2_e'] == 10.0):
    M9_med['EX_o2_e'] = 20.0

# from simulation with refinegems
M9_med['EX_cobalt2_e'] = 10.0
M9_med['EX_cu2_e'] = 10.0
M9_med['EX_fe2_e'] = 10.0
M9_med['EX_mn2_e'] = 10.0
M9_med['EX_zn2_e'] = 10.0

In [10]:
model = rg.io.load_model_cobra('../../models/Cstr_1197.xml')
M9_med['EX_ala_B_e'] = 10.0
M9_med['EX_cgly_e'] = 10.0
#M9_med['EX_glc__D_e'] = 20.0

In [20]:
find_additives(model, M9_med)

Growth rate without additives: 0.7901
Doubling time without additives: 52.64 min


,exchange,diff
87,EX_malthp_e,1.889316e+00
86,EX_malt_e,4.672051e-01
115,EX_sucr_e,4.672051e-01
122,EX_tre_e,4.672051e-01
82,EX_lcts_e,3.801638e-01
...,...,...
71,EX_gthrd_e,2.220446e-16
108,EX_salchs4fe_e,1.110223e-16
75,EX_hdca_e,1.110223e-16
76,EX_hdcea_e,1.110223e-16


In [6]:
enhanced_growth(model, M9_med, CasA_med)

Growth rate with additives: 0.6389
Doubling time with additives: 65.1 min


In [7]:
model = rg.io.load_model_cobra('../../models/Cstr_TS.xml')
M9_med.pop('EX_ala_B_e')
M9_med.pop('EX_cgly_e')

10.0

In [8]:
find_additives(model, M9_med)

Growth rate without additives: 0.6354
Doubling time without additives: 65.45 min


,exchange,diff
49,EX_malthp_e,2.314832e+00
69,EX_sucr_e,5.446664e-01
78,EX_tre_e,5.446664e-01
48,EX_malt_e,5.446664e-01
46,EX_lcts_e,4.765831e-01
...,...,...
76,EX_thym_e,6.661338e-16
44,EX_hxa_e,3.330669e-16
25,EX_enter_e,3.330669e-16
23,EX_dhptd_e,2.220446e-16


In [9]:
enhanced_growth(model, M9_med, CasA_med)

Growth rate with additives: 0.6354
Doubling time with additives: 65.45 min


In [10]:
model = rg.io.load_model_cobra('../../models/Cstr_1115.xml')
M9_med['EX_ala_B_e'] = 10.0
M9_med['EX_cgly_e'] = 10.0

In [11]:
find_additives(model, M9_med)

Growth rate without additives: 0.7651
Doubling time without additives: 54.36 min


,exchange,diff
84,EX_malthp_e,2.295264e+00
121,EX_sucr_e,5.400622e-01
129,EX_tre_e,5.400622e-01
83,EX_malt_e,5.400622e-01
77,EX_lcts_e,4.725544e-01
...,...,...
41,EX_gln__L_e,6.661338e-16
46,EX_gly_leu__L_e,6.661338e-16
45,EX_gly_glu__L_e,6.661338e-16
24,EX_aso3_e,5.551115e-16


In [12]:
enhanced_growth(model, M9_med, CasA_med)

Growth rate with additives: 0.7651
Doubling time with additives: 54.36 min


In [16]:
model = rg.io.load_model_cobra('../../models/Cstr_1116.xml')
M9_med['EX_nmn_e'] = 10.0
M9_med['EX_s_e'] = 10.0
#M9_med.pop('EX_cgly_e')
M9_med

{'EX_ca2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_h_e': 10.0,
 'EX_k_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_na1_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_o2_e': 20.0,
 'EX_pi_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_cobalt2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_ala_B_e': 10.0,
 'EX_nmn_e': 10.0,
 'EX_lys__L_e': 10.0,
 'EX_his__L_e': 10.0,
 'EX_arg__L_e': 10.0,
 'EX_asp__L_e': 10.0,
 'EX_thr__L_e': 10.0,
 'EX_glu__L_e': 10.0,
 'EX_gly_e': 10.0,
 'EX_val__L_e': 10.0,
 'EX_met__L_e': 10.0,
 'EX_ile__L_e': 10.0,
 'EX_leu__L_e': 10.0,
 'EX_tyr__L_e': 10.0,
 'EX_phe__L_e': 10.0,
 'EX_s_e': 10.0}

In [17]:
find_additives(model, M9_med)

Growth rate without additives: 0.7593
Doubling time without additives: 54.77 min


,exchange,diff
119,EX_malthp_e,2.303414e+00
150,EX_raffin_e,8.143684e-01
118,EX_malt_e,5.436328e-01
172,EX_tre_e,5.436328e-01
164,EX_sucr_e,5.436328e-01
...,...,...
179,EX_ni2_e,8.215650e-14
180,EX_btn_e,8.215650e-14
181,EX_34dhbz_e,8.215650e-14
41,EX_anhgm_e,7.815970e-14


In [14]:
enhanced_growth(model, M9_med, CasA_med)

Growth rate with additives: 0.7301
Doubling time with additives: 56.96 min
